In [18]:
from scamp import test_run, Session, wait
from scamp_extensions.pitch import Scale
import random
import time
from pythonosc import udp_client
from config import ARTIFACTS_RELPATH
from scamp_utils import set_default_scamp_settings
from connection.touchosc import TOUCHOSC_CONNECTION_PROPS, test_run
from random import random, choice, randrange

set_default_scamp_settings()

## Using Scamp's osc methods

In [4]:
session = Session().run_as_server()
piano = session.new_osc_part(**TOUCHOSC_CONNECTION_PROPS)

In [32]:
test_run(session, piano, 50, 70, 3, [0, 4], 5)

Running sequence:  [50, 53, 56, 59, 62, 65, 68, 70, 67, 64, 61, 58, 55, 52]
Repeat:  0
Repeat:  1
Repeat:  2
Repeat:  3
Repeat:  4


## Scamp tutorial series

This section starts the scamp tutorial practice once again

In [5]:
session.tempo = 30
for pitch in (48, 53, 67):
    piano.play_note(pitch, 127, 1.0, blocking=False)
    wait(1.0)

In [4]:
session = Session().run_as_server()
piano = session.new_osc_part(**TOUCHOSC_CONNECTION_PROPS)

interval = 1
piano_pitch = 48
forte_piano = 127
diminuendo = 50
low = 36
high = 60

note_choices = [0.1, 0.2, 0.5, 1, 1.5, 2, 3]
rest_choices = [0.1, 0.2, 0.5, 1]
bar_lines = []

session.tempo = 120
while session.time() < 30:
    if random() < 0.7:
        piano.play_note(piano_pitch, forte_piano, choice(note_choices))
        wait(choice(rest_choices))
    else:
        bar_lines.append(session.time())
        piano.play_note(piano_pitch, diminuendo, choice(note_choices))
        wait(choice(rest_choices))

    piano_pitch = (piano_pitch + interval - low) % (high - low) + low
    interval += 1

## Basic drum loop

In [57]:
session = Session().run_as_server()
drums = session.new_osc_part(**TOUCHOSC_CONNECTION_PROPS)

session.tempo = 60
SESSION_DURATION = 10

SNARE_CODE = 40
KICK_CODE = 36
CYMBAL_CODE = 32
HIHAT_CODE = 25


def fff():
    return 122 + randrange(0, 5)


def mf():
    return 100 + randrange(0, 5)


def double_kick():
    beat = 1
    while session.time() < SESSION_DURATION:
        if beat % 4 == 1:
            velocity = fff()
        else:
            velocity = mf()
        drums.play_note(KICK_CODE, velocity, 0.125)
        beat += 1


def snare():
    beat = 1
    while session.time() < SESSION_DURATION:
        if beat % 8 == 0:
            wait(0.50)
            drums.play_note(SNARE_CODE, mf(), 0.1, blocking=False)
            wait(0.25)
            drums.play_note(SNARE_CODE, fff(), 0.1, blocking=False)
            wait(0.125)
            drums.play_note(SNARE_CODE, mf(), 0.1, blocking=False)
            wait(0.125)
        else:
            wait(0.5)
            drums.play_note(SNARE_CODE, mf(), 0.1, blocking=False)
            wait(0.5)
        beat += 1


def rest():
    beat = 1
    while session.time() < SESSION_DURATION:
        if beat % 4 == 0:
            kick_velocity = 1
            snare_velocity = 0.8
        else:
            kick_velocity = 0.5
            snare_velocity = 0.5

        if beat % 4 == 1:
            drums.play_note(CYMBAL_CODE, fff(), 0.1, blocking=False)

        drums.play_note(HIHAT_CODE, mf(), 0.1, blocking=False)
        wait(0.5)
        drums.play_note(HIHAT_CODE, mf(), 0.1, blocking=False)
        wait(0.5)
        beat += 1


session.fork(double_kick)
session.fork(snare)
session.fork(rest)

Clock('rest')[Clock('DO_PLAY_NOTE')[], Clock('DO_PLAY_NOTE')[]]